In [3]:
def random_prime(a, b):
    while True:
        p = ZZ.random_element(2**k, 2**(k+1))
        if p.is_prime() == True:
            return p
 

class Shamir():
    
    def __init__(self, t, n):
        self._t = t
        self._n = n

        
    def GC(self, k):
        ## (a) pick a prime p
        p = random_prime(2**k, 2**(k+1))
        assert p > self._n
        
        ## (b) pick uniformly at random n distinct non-zero elements alpha from Zp*
        Zp = Zmod(p)
        alpha = []
        for i in range(self._n):
            alpha.append(Zp.random_element())
        
        return (p, alpha)
        # return x=(p, alpha)
        
        
    def DS(self, p, alpha, s):
        ## generate t-1 uniformly random elements a=(a_1,...,a_(t-1)) from Zp
        Zp = Zmod(p)
        a = [s]
        for i in range(self._t - 1):
            a.append(Zp.random_element()) 
            
        # build the polynomial a(x)=s+a_1*x+...+a_(t-1)*x^(t-1) from Zp[x;t-1]
        F = FiniteField(p)
        P = F['x']
        ax = P(a)
        
        # generate shares sigma_i
        shares = []
        for i in range(self._n):
            sigma_i = ax(alpha[i]) % p
            shares.append([alpha[i], sigma_i])
        return shares
        
        
    def SC(self, p, shares):
        assert len(shares) == self._t
        F = FiniteField(p)
        P = F['x']
        print (P.lagrange_polynomial(shares))
        
        
s = 100
k = 33

shaTSS = Shamir(4, 6)
p, alpha = shaTSS.GC(k)
shares = shaTSS.DS(p, alpha, s)
print (shares)
shaTSS.SC(p, shares[:3] + [[123, 123]])
shaTSS.SC(p, shares[:4])

    


[[9705749884, 7445316671], [10669656282, 3683233117], [1901281314, 4219082565], [11578120651, 11099184559], [1758908632, 472878132], [10441228395, 7797734872]]
3109581422*x^3 + 9898135628*x^2 + 5217614598*x + 11216524502
6478421261*x^3 + 2549975545*x^2 + 7016329006*x + 100
